In [2]:
import os
import pandas as pd

spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,479 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [1

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-17 04:23:13--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-03-17 04:23:13 (10.3 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
import pandas as pd
from sqlalchemy import create_engine

In [82]:
# Connect to database with SQLAlchemy
password = '<pw>'
user = 'postgres'
host = 'mypostgresdb.c3a0r7clbodu.us-east-1.rds.amazonaws.com'
db = 'my_data_class_db'
rds_connection_string = f"{user}:{password}@{host}:5432/{db}"

engine = create_engine(f'postgresql://{rds_connection_string}')

# engine.table_names()

# Then display vine_table with pandas
# pd.read_sql_query('SELECT * FROM vine_table', con=engine)

In [83]:
# Create spark df from vine_table
table = 'vine_table'
df = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{host}:5432/{db}") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .load()

df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [84]:
#display the spark df
df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8EWA1OFT84NX|          5|            0|          0|   N|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|
| RL5ESX231LZ0B|          5|            0|          0|   N|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|
|R13EPSFP5DODN5|          4|            0|          0|   N|
| R6RBP4HTE67SY|          5|            0|          0|   N|
|R15B3EU40RSU2W|          5|            0|          0|   N|
| RP4DD53A4ZJA2|          5|            0|          0|   N|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|
| REV51EW323H8W|          5|            0|          0|   N|
|R2GQ3W03WIUZKE|          5|            0|          0|   N|
| RTI1YI7K6GE3D|          5|            0|          0|   N|
|R3V9C2C0SPSZU6|          5|            

In [85]:
vine_df = df.filter(df.vine == "Y")\
  .groupBy('star_rating')\
  .agg({'vine':'count'})\
  .withColumnRenamed('count(vine)', 'Number of Vine Reviews')\
  .orderBy('star_rating')

vine_df.show()

+-----------+----------------------+
|star_rating|Number of Vine Reviews|
+-----------+----------------------+
|          1|                   134|
|          2|                   394|
|          3|                  1320|
|          4|                  4232|
|          5|                  6019|
+-----------+----------------------+



In [86]:
non_vine_df = df.filter(df.vine == "N")\
  .groupBy('star_rating')\
  .agg({'vine':'count'})\
  .withColumnRenamed('count(vine)', 'Number of Non-Vine Reviews')\
  .orderBy('star_rating')

non_vine_df.show()

+-----------+--------------------------+
|star_rating|Number of Non-Vine Reviews|
+-----------+--------------------------+
|          1|                    144257|
|          2|                    100549|
|          3|                    148604|
|          4|                    283119|
|          5|                   1064099|
+-----------+--------------------------+



In [87]:
avg_star_df = df.select('star_rating').agg({'star_rating':'mean'})
avg_star = avg_star_df.collect()[0][0]
avg_star = round(avg_star, 0)
print(f'Average star rating: {avg_star}')

Average star rating: 4.0


In [88]:
vine_review_count = vine_df.collect()[0][1] + vine_df.collect()[1][1] + vine_df.collect()[2][1] + vine_df.collect()[3][1] + vine_df.collect()[4][1]
above_avg_vine_review_count = vine_df.collect()[3][1] + vine_df.collect()[4][1]

non_vine_review_count = non_vine_df.collect()[0][1] + non_vine_df.collect()[1][1] + non_vine_df.collect()[2][1] + non_vine_df.collect()[3][1] + non_vine_df.collect()[4][1]
above_avg_non_vine_review_count = non_vine_df.collect()[3][1] + non_vine_df.collect()[4][1]

above_avg_vine_percentage = above_avg_vine_review_count / vine_review_count
print(f'Percent of vine reviews rated at least average 4 stars: {round(above_avg_vine_percentage*100, 2)}%')

above_avg_non_vine_percentage = above_avg_non_vine_review_count / non_vine_review_count
print(f'Percent of non-vine reviews rated at least average 4 stars: {round(above_avg_non_vine_percentage*100, 2)}%')


Percent of vine reviews rated at least average 4 stars: 84.73%
Percent of non-vine reviews rated at least average 4 stars: 77.4%


In [89]:
#calculate the percent of vine reviews that are 5 stars, 4 stars, etc

vine_one_star_percentage = vine_df.collect()[0][1] / vine_review_count 
vine_two_star_percentage = vine_df.collect()[1][1] / vine_review_count 
vine_three_star_percentage = vine_df.collect()[2][1] / vine_review_count 
vine_four_star_percentage = vine_df.collect()[3][1] / vine_review_count 
vine_five_star_percentage = vine_df.collect()[4][1] / vine_review_count 

print(f'% One-star Vine Reviews: %{round(vine_one_star_percentage * 100, 2)}')
print(f'% Two-star Vine Reviews: %{round(vine_two_star_percentage * 100, 2)}')
print(f'% Three-star Vine Reviews: %{round(vine_three_star_percentage * 100, 2)}')
print(f'% Four-star Vine Reviews: %{round(vine_four_star_percentage * 100, 2)}')
print(f'% Five-star Vine Reviews: %{round(vine_five_star_percentage * 100, 2)}')




% One-star Vine Reviews: %1.11
% Two-star Vine Reviews: %3.26
% Three-star Vine Reviews: %10.91
% Four-star Vine Reviews: %34.98
% Five-star Vine Reviews: %49.75


In [90]:
#calculate the percent of vine reviews that are 5 stars, 4 stars, etc

non_vine_one_star_percentage = non_vine_df.collect()[0][1] / non_vine_review_count 
non_vine_two_star_percentage = non_vine_df.collect()[1][1] / non_vine_review_count 
non_vine_three_star_percentage = non_vine_df.collect()[2][1] / non_vine_review_count 
non_vine_four_star_percentage = non_vine_df.collect()[3][1] / non_vine_review_count 
non_vine_five_star_percentage = non_vine_df.collect()[4][1] / non_vine_review_count 

print(f'% One-star Non_Vine Reviews: %{round(non_vine_one_star_percentage * 100, 2)}')
print(f'% Two-star Non_Vine Reviews: %{round(non_vine_two_star_percentage * 100, 2)}')
print(f'% Three-star Non_Vine Reviews: %{round(non_vine_three_star_percentage * 100, 2)}')
print(f'% Four-star Non_Vine Reviews: %{round(non_vine_four_star_percentage * 100, 2)}')
print(f'% Five-star Non_Vine Reviews: %{round(non_vine_five_star_percentage * 100, 2)}')

% One-star Non_Vine Reviews: %8.29
% Two-star Non_Vine Reviews: %5.78
% Three-star Non_Vine Reviews: %8.54
% Four-star Non_Vine Reviews: %16.27
% Five-star Non_Vine Reviews: %61.13


84.73% of vine reviews  scored at or above average (at least 4 stars) where as only 77.4% of non-vine reviews scored 4 or more stars.